### Extract additional information about albums and artists from Spotify API

In [4]:
# Import Spotipy python library
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials #To access authorised Spotify data

client_id = '43e88b2cafb14d69b6938733e86241bf'
client_secret = '7ac18034084f457eb4dd9fc9994cb416'

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager) #spotify object to access API

In [5]:
import pandas as pd
import numpy as np
import re

In [ ]:
# Import module containing functions for extracting information from API
import * from spotify

In [ ]:
# Unpickle list of album titles scraped from AOTY
import pickle

infile3 = open('titles','rb')
album_titles = pickle.load(infile3)
infile3.close()

In [ ]:
# Unpickle list of artist titles scraped from AOTY
infile = open('artists','rb')
artists = pickle.load(infile)
infile.close()

In [58]:
# Unpickle merged dataframe containing scraped data from AOTY 
infile = open('/Users/sunnajo/metis/onl20_ds4/projects/project2/data/merged','rb')
no_sp_df = pickle.load(infile)
infile.close()

no_sp_df.head()

,title,artist,release_date,format,major_label,genres,num_tracks,features_others,debut_album,num_user_ratings,critic_score,artist_critic_score,artist_user_score,artist_num_user_ratings,years_active,user_score
0,To Pimp a Butterfly,Kendrick Lamar,"March 16, 2015",1,1,Hip Hop,16.0,1.0,0,4504,95.0,89.0,87,13780.0,17.0,93
1,"good kid, m.A.A.d. city",Kendrick Lamar,"October 22, 2012",1,0,Hip Hop,11.0,1.0,0,3409,90.0,89.0,87,13780.0,17.0,91
2,DAMN.,Kendrick Lamar,"April 14, 2017",1,0,Hip Hop,14.0,1.0,0,3695,93.0,89.0,87,13780.0,17.0,82
3,Section.80,Kendrick Lamar,"July 2, 2011",1,0,Hip Hop,16.0,0.0,1,1590,80.0,89.0,87,13780.0,17.0,81
4,Kendrick Lamar EP,Kendrick Lamar,"December 29, 2009",0,0,Hip Hop,15.0,1.0,0,106,70.0,89.0,87,13780.0,17.0,73


In [446]:
# Create dataframe of album titles and artists
aa = df[['title', 'artist']]
aa.head()

,title,artist
0,To Pimp a Butterfly,Kendrick Lamar
1,"good kid, m.A.A.d. city",Kendrick Lamar
2,DAMN.,Kendrick Lamar
3,Section.80,Kendrick Lamar
4,Kendrick Lamar EP,Kendrick Lamar


In [447]:
# Create lists of albums titles and artists
albums = list(aa['title'])
artists = list(aa['artist'])

**Spotify API pipeline:**

1) Loop through album titles and query each album    
2) Loop through album ids and get additional info about each album    
3) Loop through tracks and get features of each track    
4) Compile track information to get additional info about corresponding album    
5) Create dictionary for each album with its features    
6) Get additional artist information

In [473]:
# Create list of dictionaries containing specified features for each album obtained from Spotify API
sp_list = [get_album_features(album) for album in albums]
len(sp_list)

1303

In [621]:
# Convert into dataframe
sp_df = pd.DataFrame(sp_list)
sp_df.head()

,title,artist,total_tracks,length,explicit,mode,avg_tempo,instrumentalness,album_sp_pop,artist_genres,artist_sp_pop,feat,feat_mult
0,To Pimp A Butterfly,Kendrick Lamar,16.0,4739.612,16.0,8.0,86.935,0.000173,78.0,"[conscious hip hop, hip hop, rap, west coast rap]",90.0,1.0,1.0
1,"good kid, m.A.A.d city",Kendrick Lamar,13.0,4380.266,13.0,7.0,133.798,0.011372,76.0,"[conscious hip hop, hip hop, rap, west coast rap]",90.0,1.0,0.0
2,DAMN.,Kendrick Lamar,14.0,3300.973,14.0,8.0,174.516,0.085985,86.0,"[conscious hip hop, hip hop, rap, west coast rap]",90.0,1.0,0.0
3,Section.80,Kendrick Lamar,15.0,3272.577,15.0,7.0,81.024,0.008681,73.0,"[conscious hip hop, hip hop, rap, west coast rap]",90.0,1.0,0.0
4,Kendrick Lamar EP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [633]:
# Create dataframe containing non-null values to be able to extract genres
# from 'artist_genres' column
sp_df_nona = sp_df.dropna()
sp_df_nona.isna().sum()

title               0
artist              0
total_tracks        0
length              0
explicit            0
mode                0
avg_tempo           0
instrumentalness    0
album_sp_pop        0
artist_genres       0
artist_sp_pop       0
feat                0
feat_mult           0
dtype: int64

In [651]:
# Explore 'artist_genres' column
# Create list of genres from this column
genres = []

for idx, row in sp_df_nona.iterrows():
    if len(row['artist_genres']) > 0:
        genres = [genre for genre in row['artist_genres']]

genres

['conscious hip hop',
 'hip hop',
 'rap',
 'west coast rap',
 'conscious hip hop',
 'hip hop',
 'rap',
 'west coast rap',
 'conscious hip hop',
 'hip hop',
 'rap',
 'west coast rap',
 'conscious hip hop',
 'hip hop',
 'rap',
 'west coast rap',
 'conscious hip hop',
 'hip hop',
 'rap',
 'west coast rap',
 'alternative hip hop',
 'hip hop',
 'psychedelic hip hop',
 'rap',
 'conscious hip hop',
 'east coast hip hop',
 'gangster rap',
 'hardcore hip hop',
 'hip hop',
 'queens hip hop',
 'rap',
 'conscious hip hop',
 'east coast hip hop',
 'gangster rap',
 'hardcore hip hop',
 'hip hop',
 'queens hip hop',
 'rap',
 'conscious hip hop',
 'east coast hip hop',
 'gangster rap',
 'hardcore hip hop',
 'hip hop',
 'queens hip hop',
 'rap',
 'conscious hip hop',
 'east coast hip hop',
 'gangster rap',
 'hardcore hip hop',
 'hip hop',
 'queens hip hop',
 'rap',
 'atl hip hop',
 'pop rap',
 'rap',
 'southern hip hop',
 'trap',
 'conscious hip hop',
 'east coast hip hop',
 'gangster rap',
 'hardcore 

In [661]:
# Look at most common genres
from collections import Counter

genres_c = Counter(genres)
genres_c.most_common()

[('rap', 641),
 ('hip hop', 591),
 ('pop rap', 340),
 ('trap', 275),
 ('alternative hip hop', 267),
 ('underground hip hop', 258),
 ('southern hip hop', 246),
 ('gangster rap', 169),
 ('escape room', 139),
 ('east coast hip hop', 134),
 ('conscious hip hop', 133),
 ('hardcore hip hop', 126),
 ('pop', 101),
 ('melodic rap', 100),
 ('atl hip hop', 91),
 ('vapor trap', 66),
 ('dance pop', 62),
 ('chicago rap', 55),
 ('emo rap', 55),
 ('experimental hip hop', 45),
 ('boom bap', 43),
 ('dirty south rap', 40),
 ('queens hip hop', 39),
 ('miami hip hop', 35),
 ('psychedelic hip hop', 34),
 ('political hip hop', 33),
 ('west coast rap', 32),
 ('dark trap', 32),
 ('detroit hip hop', 30),
 ('trap queen', 30),
 ('abstract hip hop', 29),
 ('canadian hip hop', 29),
 ('philly rap', 23),
 ('hip pop', 23),
 ('toronto rap', 23),
 ('jazz rap', 22),
 ('turntablism', 22),
 ('meme rap', 22),
 ('new orleans rap', 21),
 ('industrial hip hop', 20),
 ('old school hip hop', 20),
 ('neo soul', 19),
 ('indie r&b'

In [666]:
# Look at rows in original df with all NaN values for Spotify data
indexes = [idx for idx, row in sp_feat_df[sp_feat_df['artist'].isna()].iterrows()]

In [536]:
# Check what information from merged df is missing that I can potentially fill in with data from Spotify
df.isna().sum()

# num_tracks, features_others

title                        0
artist                       0
release_date                 0
format                       0
major_label                  0
genres                       0
num_tracks                  91
features_others             91
debut_album                  0
num_user_ratings             0
critic_score               168
artist_critic_score         75
artist_user_score            0
artist_num_user_ratings      4
years_active                62
user_score                   0
dtype: int64

In [543]:
# Look at 'genres' column
sp_df[sp_df["genres"].str.len() != 0]

# This column contains either empty list or NaN

,title,artist,label,total_tracks,genres,length,explicit,mode,avg_tempo,instrumentalness,album_sp_pop,artist_genres,artist_sp_pop
4,Kendrick Lamar EP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62,Steroids (Crouching Tiger Hidden Gabber Megamix),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
63,Niggas on the Moon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82,Stress: The Extinction Agenda,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
111,Funcrusher Plus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
122,Third Side of Tape,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
123,Three Sided Tape Volume One,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
126,Only Built 4 Cuban Linx... Pt. II,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
129,Venomous Villain,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
215,3 Feet High and Rising,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [544]:
# Drop columns: label (duplicate info as not missing any information in data scraped from AOTY),
# genres (contains empty lists)

sp_df.drop(columns=['label', 'genres'], inplace=True)
sp_df.head()

,title,artist,total_tracks,length,explicit,mode,avg_tempo,instrumentalness,album_sp_pop,artist_genres,artist_sp_pop
0,To Pimp A Butterfly,Kendrick Lamar,16.0,4739.612,16.0,8.0,86.935,0.000173,78.0,"[conscious hip hop, hip hop, rap, west coast rap]",90.0
1,"good kid, m.A.A.d city",Kendrick Lamar,13.0,4380.266,13.0,7.0,133.798,0.011372,76.0,"[conscious hip hop, hip hop, rap, west coast rap]",90.0
2,DAMN.,Kendrick Lamar,14.0,3300.973,14.0,8.0,174.516,0.085985,86.0,"[conscious hip hop, hip hop, rap, west coast rap]",90.0
3,Section.80,Kendrick Lamar,15.0,3272.577,15.0,7.0,81.024,0.008681,73.0,"[conscious hip hop, hip hop, rap, west coast rap]",90.0
4,Kendrick Lamar EP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [566]:
# Check if rows with null values in merged df have information in Spotify df
df_numtracks_na_idx = list(np.where(df['num_tracks'].isna())[0])

In [568]:
len(df_numtracks_na_idx)

91

In [674]:
numtracks_l = [sp_feat_df.iloc[idx] for idx in df_numtracks_na_idx]
len(numtracks_l)

91

In [675]:
# Make new df with num_tracks info
numtracks_df = pd.DataFrame(numtracks_l)

In [677]:
numtracks_df.head()

,title,artist,total_tracks,length,explicit,mode,avg_tempo,instrumentalness,album_sp_pop,artist_genres,artist_sp_pop,feat,feat_mult
71,The Shining,J Dilla,12.0,2191.013,11.0,6.0,94.120,2.177175,55.0,"[alternative hip hop, detroit hip hop, hip hop]",62.0,1.0,1.0
91,The Dynasty: Roc La Famila 2000,JAY-Z,16.0,4005.334,0.0,6.0,81.566,0.088638,29.0,"[east coast hip hop, hip hop, pop rap, rap]",87.0,1.0,1.0
157,The Main Ingredient,Pete Rock & C.L. Smooth,16.0,4570.747,6.0,13.0,89.570,0.410309,44.0,"[alternative hip hop, east coast hip hop, gang...",52.0,0.0,0.0
173,KIDS SEE GHOSTS,KIDS SEE GHOSTS,7.0,1432.309,7.0,4.0,110.011,0.040595,75.0,"[hip hop, rap]",67.0,1.0,0.0
174,6 Feet Deep,Gravediggaz,16.0,3018.201,0.0,7.0,177.908,0.022815,43.0,"[alternative hip hop, east coast hip hop, gang...",44.0,0.0,0.0


In [678]:
sp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1303 entries, 0 to 1302
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   title             1303 non-null   object 
 1   artist            1258 non-null   object 
 2   total_tracks      1258 non-null   float64
 3   length            1258 non-null   float64
 4   explicit          1258 non-null   float64
 5   mode              1258 non-null   float64
 6   avg_tempo         1258 non-null   float64
 7   instrumentalness  1258 non-null   float64
 8   album_sp_pop      1258 non-null   float64
 9   artist_genres     1258 non-null   object 
 10  artist_sp_pop     1258 non-null   float64
 11  feat              1259 non-null   float64
 12  feat_mult         1259 non-null   float64
dtypes: float64(10), object(3)
memory usage: 132.5+ KB


In [681]:
# Merge dataframes containing information scraped from AOTY and from Spotify
merged_sp_df = pd.merge(df, sp_df, left_index=True, right_index=True, how='outer')
merged_sp_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1303 entries, 0 to 1302
Data columns (total 29 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   title_x                  1303 non-null   object 
 1   artist_x                 1303 non-null   object 
 2   release_date             1303 non-null   object 
 3   format                   1303 non-null   int64  
 4   major_label              1303 non-null   int64  
 5   genres                   1303 non-null   object 
 6   num_tracks               1212 non-null   float64
 7   features_others          1212 non-null   float64
 8   debut_album              1303 non-null   int64  
 9   num_user_ratings         1303 non-null   int64  
 10  critic_score             1135 non-null   float64
 11  artist_critic_score      1228 non-null   float64
 12  artist_user_score        1303 non-null   object 
 13  artist_num_user_ratings  1299 non-null   float64
 14  years_active            

In [684]:
merged_sp_df.head()

,title_x,artist_x,release_date,format,major_label,genres,num_tracks,features_others,debut_album,num_user_ratings,...,length,explicit,mode,avg_tempo,instrumentalness,album_sp_pop,artist_genres,artist_sp_pop,feat,feat_mult
0,To Pimp a Butterfly,Kendrick Lamar,"March 16, 2015",1,1,Hip Hop,16.0,1.0,0,4504,...,4739.612,16.0,8.0,86.935,0.000173,78.0,"[conscious hip hop, hip hop, rap, west coast rap]",90.0,1.0,1.0
1,"good kid, m.A.A.d. city",Kendrick Lamar,"October 22, 2012",1,0,Hip Hop,11.0,1.0,0,3409,...,4380.266,13.0,7.0,133.798,0.011372,76.0,"[conscious hip hop, hip hop, rap, west coast rap]",90.0,1.0,0.0
2,DAMN.,Kendrick Lamar,"April 14, 2017",1,0,Hip Hop,14.0,1.0,0,3695,...,3300.973,14.0,8.0,174.516,0.085985,86.0,"[conscious hip hop, hip hop, rap, west coast rap]",90.0,1.0,0.0
3,Section.80,Kendrick Lamar,"July 2, 2011",1,0,Hip Hop,16.0,0.0,1,1590,...,3272.577,15.0,7.0,81.024,0.008681,73.0,"[conscious hip hop, hip hop, rap, west coast rap]",90.0,1.0,0.0
4,Kendrick Lamar EP,Kendrick Lamar,"December 29, 2009",0,0,Hip Hop,15.0,1.0,0,106,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [683]:
# Pickle new merged dataframe
outfile = open('merged_sp_df','wb')
pickle.dump(merged_sp_df,outfile)
outfile.close()

In [690]:
# Look for any errors by matching album titles
aoty_titles = list(merged_sp_df['title_x'])
spotify_titles = list(merged_sp_df['title_y'])
a_zip = zip(aoty_titles, spotify_titles)
zipped_l = list(a_zip)

In [728]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [738]:
for i in range(1304):
    if similar(aoty_titles[i].lower(), spotify_titles[i].lower()) < 0.8:
        unmatched.append([i, aoty_titles[i], spotify_titles[i]])

[[102, 'EARL', 'Early Morning Shakes'],
 [103, 'FEET OF CLAY', 'FEET OF CLAY (Deluxe)'],
 [105,
  'The Ecstatic',
  'World Spirituality Classics 1: The Ecstatic Music of Alice Coltrane'],
 [113, 'Operation: Doomsday', 'Operation: Doomsday (Complete)'],
 [115, 'Labcabincalifornia', 'Labcabincalifornia (Deluxe Edition)'],
 [136,
  "Paul's Boutique",
  "Paul's Boutique (20th Anniversary Remastered Edition)"],
 [140, 'Hello Nasty', 'Hello Nasty (Deluxe Version/Remastered 2009)'],
 [153, 'Desire', 'Desireless'],
 [166, 'Bastard', 'The Bastards'],
 [168, 'Goblin', 'Goblin (Deluxe Edition)'],
 [181, 'Death Certificate', 'Death Certificate (25th Anniversary Edition)'],
 [186, 'Be', 'beerbongs & bentleys'],
 [190, "Nobody's Smiling", "Nobody's Smiling (Deluxe)"],
 [193, 'The Cold Vein', 'The Cold Vein (Deluxe Edition)'],
 [194, 'Blade of the Ronin', 'Blade of the Ronin (Deluxe Edition)'],
 [202, 'Infinite', 'Infinitely Ordinary'],
 [203, 'Relapse', 'Relapse: Refill'],
 [212, 'Skelethon', 'Skele

In [811]:
# Errors in album titles:
unmatched_idx = [102, 105, 153, 166, 186, 202, 214, 223, 224, 228, 270, 285, 328, 378, 381, 394, 405, 407, 409, 410, 423,
 433, 445, 465, 478, 479, 483, 487, 518, 525, 535, 537, 545, 564, 571, 602, 672, 675, 677, 678, 704, 751, 759,
 784, 807, 830, 840, 853, 867, 901, 932, 937, 994, 1000, 1033, 1050, 1123, 1130, 1165, 1179, 1181, 1188, 1204, 1269,
 1296]

In [ ]:
unmatched = [(idx, albums[idx], artists[idx]) for idx in unmatched_idx]

In [45]:
# Fixing errors in album titles
# Create list of lists containing df index + album ids OR dictionary containing features via manual input,
# then run function to extract features

err1_l = [
[13, '4UhEjfIRx4tE1XRY21vwNa'],
[15, '5dMmFu9NCxSjDk3ekggogc'],
[21, '2WU4hhnp2kjPckmjqdL1RT'],
[53, '3vyMhFtK8R0cFoMfnBfztb'],
[59, '2WIJUm8je4Pz033J9pP2uA'],
[66, {
        'title': 'Death Grips',
        'artist': 'Death Grips',
        'total_tracks': 6,
        'feat': 0,
        'feat_mult': 0,
        'genres': 'experimental hip hop',
        'length': 1503,
        'artist_genres': ['alternative hip hop',
 'escape room',
 'experimental hip hop',
 'hip hop',
 'industrial hip hop'],
    }],
[84, '69CmkikTHkGKdkrUZTtyWl'],
[102, {
        'title': 'Earl',
        'artist': 'Earl Sweatshirt',
        'total_tracks': 10,
        'feat': 1,
        'feat_mult': 1,
        'genres': ['alternative hip hop', 'horrorcore'],
        'length': 1554,
        'artist_genres': ['alternative hip hop',
 'escape room',
 'experimental hip hop',
 'hip hop',
 'rap',
 'underground hip hop'],
    }],
[105, '2Q8ugdrlhqWh3jJ7b6oIjY'],
[153, '0XxYd2lssGlf1jzO77HPcY'],
[166, {
        'title': 'Bastard',
        'artist': 'Tyler, the Creator',
        'total_tracks': 15,
        'feat': 1,
        'feat_mult': 1,
        'genres': 'alternative hip hop',
        'length': 3305
    }],
[186, '2UuvBxV56QWWj2uviGS0up'],
[202, {
        'title': 'Infinite',
        'artist': 'Eminem',
        'label': 'Web Entertainment',
        'total_tracks': 11,
        'feat': 1,
        'feat_mult': 1,
        'genres': 'hip hop',
        'length': 2274,
        'artist_genres': ['detroit hip hop', 'hip hop', 'rap'],
    }],
[214, '4uC2NmWBxzizUR5Q2y2IAg'],
[223, '6IH1d0LM9ITTPY1BTSXolc'],
[224, '7sZqZpGN6leR2PncwkuaUc'],
[228, {
        'title': '4eva N a Day',
        'artist': 'Big K.R.I.T.',
        'label': 'Cinematic Music Group',
        'total_tracks': 17,
        'feat': 0,
        'feat_mult': 0,
        'genres': ['hip hop', 'southern hip hop'],
        'length': 3005,
        'artist_genres': ['alternative hip hop',
 'conscious hip hop',
 'dirty south rap',
 'gangster rap',
 'hip hop',
 'mississippi hip hop',
 'rap',
 'southern hip hop',
 'trap',
 'underground hip hop']
    }],
[270, '5qzulzfu4wC4MMccjQFgtK'],
[285, '3UesepjW7Scwi8DV62Qqyn'],
[328, '0TDJRkEr2SrhWTetdkEzED'],
[378, '2pWiCJOBqxq4jtmuvdQG4J'],
[381, '60Ca2ECXMePq5zJ0fjcQK1'],
[394, {
        'title': 'BetterOffDEAD',
        'artist': 'Flatbush Zombies',
        'label': 'ElectricKoolAde Records',
        'total_tracks': 19,
        'feat': 1,
        'feat_mult': 1,
        'genres': 'hip hop',
        'length': 3951,
        'artist_genres': ['escape room', 'hip hop', 'psychedelic hip hop', 'rap', 'underground hip hop']}],
[405, '1HB4naIXCKcUtvBK4Gmcke'],
[407, {
        'title': "Hell's Winter",
        'artist': 'Cage',
        'total_tracks': 14,
        'feat': 1,
        'feat_mult': 1,
        'genres': 'hip hop',
        'length': 3308,
        'artist_genres': ['abstract hip hop'],
    }],
[409, '0apVddbk7Juec0DV4LNiBg'],
[410, '11KgjDEaT1YjNUphDDTOzc'],
[423, {
        'title': 'Live. Love. A$AP',
        'artist': 'A$AP Rocky',
        'total_tracks': 16,
        'feat': 1,
        'feat_mult': 1,
        'genres': 'hip hop',
        'length': 3221,
        'artist_genres': ['east coast hip hop', 'hip hop', 'rap', 'southern hip hop', 'trap'],
    }],
[433, {
        'title': "Pilot Talk",
        'artist': 'Curren$y',
        'total_tracks': 13,
        'feat': 1,
        'feat_mult': 1,
        'genres': 'hip hop',
        'length': 2569,
        'artist_genres': 'abstract hip hop',
    }],
[445, '5POcKy926GgzFHZpGptJac'],
[465, '5yXvXFTwuqoAHFuPeliqvZ'],
[478, '65hRa80KDworqyE8242rLN'],
[479, '6vmsGTuTYTo5GDHkosmq2E'],
[483, '17FBoXK1NU2rvJBbzdzw0r'],
[487, '5u8ri4s76Ew14IwGOJf5hI'],
[518, '3KhH1DkflxVkR9ABWV7rNG'],
[525, '602bFcdb1Fof8HImMWxYvp'],
[535, '4DB0I6kc8VpP6mbdt0V2Ui'],
[537, '05n0d2kfwGPhKpTonLHRpY'],
[545, '4W5qCJNSN3oNHB986TdO9S'],
[564, '4PishS8bjccWUr4CVrosmb'],
[571, '0Wh0QKnKq6j7Tlkj7RYE9u'],
[602, {
        'title': 'BBNG',
        'artist': 'BADBADNOTGOOD',
        'total_tracks': 12,
        'feat': 0,
        'feat_mult': 0,
        'genres': ['jazz', 'instrumental hip hop', 'post-bop', 'electronica'],
        'length': 2787,
        'artist_genres': 'jazz instrumental hip hop',
    }],
[672, '1dSvdsbgLtbSd7vUckHvKS'],
[675, '6L4gtDcKB0RBRpx2DhHrsQ'],
[677, {
        'title': 'Culdesac',
        'artist': 'Childish Gambino',
        'total_tracks': 16,
        'feat': 1,
        'feat_mult': 0,
        'genres': ['hip hop', 'indie pop'],
        'length': 3386,
        'artist_genres': ['atl hip hop', 'hip hop', 'pop rap', 'rap'],
    }],
[678, {
        'title': 'STN MTN',
        'artist': 'Childish Gambino',
        'total_tracks': 11,
        'feat': 1,
        'feat_mult': 1,
        'genres': 'hip hop',
        'length': 2428,
        'artist_genres': ['atl hip hop', 'hip hop', 'pop rap', 'rap'],
    }],
[704, '1HiN2YXZcc3EjmVZ4WjfBk'],
[751, '0cMxALtiABnT1kIuA1wgsQ'],
[759, '6pbfofGywFkDyt1HoKYnRd'],
[784, '0MTTE1UtmLcN9Tm2GooEfZ'],
[807, '63Pn7ir6TrMHAVYVNMfTbN'],
[830, '6sWz3hjGjLYCtcnPPuRe9i'],
[840, '64vVGhv3dOHZxWaRJTHvTI'],
[853, {
        'title': 'Lil Uzi Vert vs. the World 2',
        'artist': 'Lil Uzi Vert',
        'total_tracks': 14,
        'feat': 1,
        'feat_mult': 1,
        'genres': 'hip hop',
        'length': 2564,
        'artist_genres': ['melodic rap', 'philly rap', 'rap', 'trap'],
    }],
[867, '1wQ8SUjCs4OqmWnCzi6oI9'],
[901, '21sofiD2YYtvZmeYKVjPf8'],
[932, '6UISNuckF9Fgj9PGR4dYcm'],
[937, '5971dUyMLcsZa7IBioI7lP'],
[994, {
        'title': 'Dedication 6',
        'artist': 'Lil Wayne',
        'total_tracks': 15,
        'feat': 1,
        'feat_mult': 1,
        'genres': 'hip hop',
        'length': 3272,
        'artist_genres': ['hip hop', 'new orleans rap', 'pop rap', 'rap', 'trap'],
    }],
[1000, {
        'title': '1983',
        'artist': 'Flying Lotus',
        'total_tracks': 11,
        'feat': 0,
        'feat_mult': 0,
        'genres': ['trip hop', 'wonky'],
        'length': 2039,
    }],
[1033, '7bhh6bvHq7hqc2VQFgrwE5'],
[1050, '6qj0FWwNsaGtR8Eo0Qx4Wt'],
[1123, '41tn9gcVocI693HtBidKfi'],
[1130, '7Ja0MmZhnVdFjs5QOPj1lj'],
[1165, '1Mp3A4tFDAiaMeGFJUzo1c'],
[1179, '4BvTosEiFqdaPftdFdbUzT'],
[1181, {
        'title': 'Jxmtro',
        'artist': 'Slim Jxmmi',
        'total_tracks': 9,
        'feat': 1,
        'feat_mult': 1,
        'genres': 'trap',
        'length': 1693,
    }],
[1188, '4IRiXE5NROxknUSAUSjMoO'],
[1204, null_d],
[1269, {
        'title': 'Brown Boy 2',
        'artist': 'NAV',
        'total_tracks': 14,
        'feat': 1,
        'feat_mult': 1,
        'genres': ['hip hop', 'trap'],
        'length': 2260,
    }],
[1296, '2ac61RS9in31ckWVBHvoYw']
]

In [46]:
# Second round of error checks
err2_l = [
    [11, '0ddTtajzPmxZ7UVW2VEAln'],
    [188, '0JKjuvDUPrnlblAZyB1Aje'],
    [233, {'title': "It's Better This Way",
 'artist': 'Big K.R.I.T.',
 'total_tracks': 14,
 'feat': 1,
 'feat_mult': 1,
 'length': 2877,
 'explicit': np.nan,
 'mode': np.nan,
 'avg_tempo': np.nan,
 'instrumentalness': np.nan,
 'album_sp_pop': np.nan,
 'artist_genres': ['alternative hip hop',
 'conscious hip hop',
 'dirty south rap',
 'gangster rap',
 'hip hop',
 'mississippi hip hop',
 'rap',
 'southern hip hop',
 'trap',
 'underground hip hop'],
 'artist_sp_pop': 66}],
    [237, {'title': 'Faces',
 'artist': 'Mac Miller',
 'total_tracks': 24,
 'feat': 1,
 'feat_mult': 1,
 'length': 5138,
 'explicit': np.nan,
 'mode': np.nan,
 'avg_tempo': np.nan,
 'instrumentalness': np.nan,
 'album_sp_pop': np.nan,
 'artist_genres': ['hip hop, pittsburgh rap, rap'],
 'artist_sp_pop': 87}],
    [248, '3DXcW4NMarpCjpP628mX4j'],
    [318, '6UyA8TFJ1KrbYJolngGUe5'],
    [380, '6n6SDRsK93ExHgJinmpdSo'],
    [440, {'title': 'Da Drought 3',
 'artist': 'Lil Wayne',
 'total_tracks': 29,
 'feat': 1,
 'feat_mult': 1,
 'length': 6486,
 'explicit': np.nan,
 'mode': np.nan,
 'avg_tempo': np.nan,
 'instrumentalness': np.nan,
 'album_sp_pop': np.nan,
 'artist_genres': ['hip hop', 'new orleans rap', 'pop rap', 'rap', 'trap'],
 'artist_sp_pop': 92}],
    [449, {'title': 'Sorry 4 the Wait 2',
 'artist': 'Lil Wayne',
 'total_tracks': 18,
 'feat': 1,
 'feat_mult': 1,
 'length': 3901,
 'explicit': np.nan,
 'mode': np.nan,
 'avg_tempo': np.nan,
 'instrumentalness': np.nan,
 'album_sp_pop': np.nan,
 'artist_genres': ['hip hop', 'new orleans rap', 'pop rap', 'rap', 'trap'],
 'artist_sp_pop': 92}],
    [485, '5rbqupVSYwvEmXBUnFtUf7'],
    [570, '25Lia0Q4UWkAqIBF30QVJE'],
    [577, '1Nel9Gb0ieXLZ8kx8hHnoM'],
    [699, {'title': 'Young Sinatra',
 'artist': 'Logic',
 'total_tracks': 22,
 'feat': 1,
 'feat_mult': 1,
 'length': 4362,
 'explicit': np.nan,
 'mode': np.nan,
 'avg_tempo': np.nan,
 'instrumentalness': np.nan,
 'album_sp_pop': np.nan,
 'artist_genres': ['conscious hip hop', 'dmv rap', 'hip hop', 'pop rap', 'rap'],
 'artist_sp_pop': 86}],
    [732, '6wi40lIjhukePWyXRKY7qO'],
    [744, '6u66sAgu3xFfKhQpBr4dSu'],
    [745, '0Un0VSFgdup23s7Yzt3LL8'],
    [817, '5pYJCnVKIXdZVySWQe4DEl'],
    [828, '5oWygXdkRb6ztMGWdzxGlU'],
    [841, '6fUnZqjU3bwBOZIsQ4kN1Q'],
    [894, '5HCBV9Epivfv54NNH7VQqA'],
    [899, '4CIhgtHyTy4FPY5XzILrfV'],
    [902, '0EmCH3tGiC9pjgIhxsqg4e'],
    [982, '7no3h67ym7597W2K9sw9Qo'],
    [1023, '0cGgwoMge4N2zpTiP2GU6d'],
    [1054, {'title': 'August 26th',
 'artist': 'Post Malone',
 'total_tracks': 10,
 'feat': 1,
 'feat_mult': 1,
 'length': 2267,
 'explicit': np.nan,
 'mode': np.nan,
 'avg_tempo': np.nan,
 'instrumentalness': np.nan,
 'album_sp_pop': np.nan,
 'artist_genres': ['dfw rap', 'melodic rap', 'rap'],
 'artist_sp_pop': 95}],
    [1116, '7wQWfHMlE9OVghvW74yfz3'],
    [1196, '1VAc77UvK5wj8ZSWCo3V2b'],
    [1271, '0uxFVxnVzRu6yg8Hs44dP0']
]

In [2]:
# Function for extracting album features for albums with errors
def get_album_err(album_id):
    # Query
    try:
        q = sp.album(album_id)
    except:
        return {'title': np.nan,
        'artist': np.nan,
        'label': np.nan,
        'total_tracks': np.nan,
        'feat': np.nan,
        'mult_feat': np.nan,
        'length': np.nan,
        'explicit': np.nan,
        'mode': np.nan,
        'avg_tempo': np.nan,
        'instrumentalness': np.nan,
        'album_sp_pop': np.nan,
        'artist_genres': np.nan,
        'artist_sp_pop': np.nan}
    
    # Extract features
    title = q['name']
    artist = q['artists'][0]['name']
    label = q['label']
    total_tracks = q['total_tracks']
    sp_pop = int(q['popularity'])
    tracks = q['tracks']['items']
    artist_id = q['artists'][0]['id']
    
    tracks_l = [get_track_features(track) for track in tracks]

    # Compile track information to determine album features
    album_length = 0
    album_exp = 0
    album_mode = 0
    album_tempo = 0
    album_inst = 0
    album_feat = 0
    album_mult_feat = 0
    
    for track_d in tracks_l:
        album_length += track_d['length']
        album_exp += track_d['exp']
        album_mode += track_d['mode']
        album_tempo = np.mean(track_d['tempo'])
        album_inst += track_d['inst']
        if track_d['feat'] > 0:
            album_feat = 1
        if track_d['num_feat'] > 1:
            album_mult_feat = 1
    
    # Get additional artist information
    artist_info = sp.artist(artist_id)
    artist_genres = artist_info['genres']   #Proxy for album genres if missing?
    artist_sp_pop = artist_info['popularity']
    
    # Create dictionary for each album
    album_d = {
        'title': title,
        'artist': artist,
        'label': label,
        'total_tracks': total_tracks,
        'feat': album_feat,
        'feat_mult': album_mult_feat,
        'length': album_length,
        'explicit': album_exp,
        'mode': album_mode,
        'avg_tempo': album_tempo,
        'instrumentalness': album_inst,
        'album_sp_pop': sp_pop,
        'artist_genres': artist_genres,
        'artist_sp_pop': artist_sp_pop
    }
    
    return album_d

In [47]:
err1_d = {}

for item in err1_l:
    if type(item[1]) == str:
        album_id = item[1]
        err1_d[item[0]] = get_album_err(album_id)
    else:
        err1_d[item[0]] = item[1]

In [48]:
# Convert corrected album information into dataframe
err1_df = pd.DataFrame(err1_d).T
err1_df.head()

,title,artist,label,total_tracks,feat,feat_mult,length,explicit,mode,avg_tempo,instrumentalness,album_sp_pop,artist_genres,artist_sp_pop,genres,mult_feat
13,I Am...,Nas,Columbia,16,1,0,3891.33,16,5,92.176,3.618e-05,64,"[conscious hip hop, east coast hip hop, gangst...",78,NaN,NaN
15,Untitled,Nas,NAS / SONY,15,1,1,3248.47,0,6,106.488,2.5e-06,21,"[conscious hip hop, east coast hip hop, gangst...",78,NaN,NaN
21,The W,Wu-Tang Clan,LOUD Records,13,1,0,3323.67,13,6,69.798,0.0106,50,"[east coast hip hop, gangster rap, hardcore hi...",71,NaN,NaN
53,XXX (Deluxe Version),Danny Brown,Fool's Gold Records,22,1,0,3669.58,22,15,136.95,0.151126,53,"[alternative hip hop, detroit hip hop, escape ...",65,NaN,NaN
59,Ex Military,Death Grips,Third Worlds,13,0,0,2913.37,11,8,96.915,0.967478,0,"[alternative hip hop, escape room, experimenta...",64,NaN,NaN


In [50]:
# Clean columns
columns = ['title', 'artist', 'total_tracks', 'length', 'explicit', 'mode',
       'avg_tempo', 'instrumentalness', 'album_sp_pop', 'artist_genres',
       'artist_sp_pop', 'feat', 'feat_mult']

err1_df = err1_df[columns]
err1_df.head()

,title,artist,total_tracks,length,explicit,mode,avg_tempo,instrumentalness,album_sp_pop,artist_genres,artist_sp_pop,feat,feat_mult
13,I Am...,Nas,16,3891.33,16,5,92.176,3.618e-05,64,"[conscious hip hop, east coast hip hop, gangst...",78,1,0
15,Untitled,Nas,15,3248.47,0,6,106.488,2.5e-06,21,"[conscious hip hop, east coast hip hop, gangst...",78,1,1
21,The W,Wu-Tang Clan,13,3323.67,13,6,69.798,0.0106,50,"[east coast hip hop, gangster rap, hardcore hi...",71,1,0
53,XXX (Deluxe Version),Danny Brown,22,3669.58,22,15,136.95,0.151126,53,"[alternative hip hop, detroit hip hop, escape ...",65,1,0
59,Ex Military,Death Grips,13,2913.37,11,8,96.915,0.967478,0,"[alternative hip hop, escape room, experimenta...",64,0,0


In [27]:
err2_d = {}

for item in err2_l:
    if type(item[1]) == str:
        album_id = item[1]
        err2_d[item[0]] = get_album_err(album_id)
    else:
        err2_d[item[0]] = item[1]

In [28]:
err2_df = pd.DataFrame(err2_d).T
err2_df.drop(columns=['label'], inplace=True)
err2_df.head()

,title,artist,total_tracks,feat,feat_mult,length,explicit,mode,avg_tempo,instrumentalness,album_sp_pop,artist_genres,artist_sp_pop
11,Life Is Good (Explicit Version),Nas,15,1,1,3746.11,15,5,88.581,0,37,"[conscious hip hop, east coast hip hop, gangst...",78
188,Resurrection,Common,15,0,0,3263.13,13,8,91.447,0.400159,55,"[alternative hip hop, chicago rap, conscious h...",68
233,It's Better This Way,Big K.R.I.T.,14,1,1,2877,NaN,NaN,NaN,NaN,NaN,"[alternative hip hop, conscious hip hop, dirty...",66
237,Faces,Mac Miller,24,1,1,5138,NaN,NaN,NaN,NaN,NaN,"[hip hop, pittsburgh rap, rap]",87
248,Absence,Dälek,10,0,0,3432.13,0,6,175.898,3.44352,18,"[abstract hip hop, conscious hip hop, dub meta...",28


In [53]:
# Combine err1 & err2 dataframes
err12_df = err1_df.append(err2_df).sort_index()
err12_df[10:26]

,title,artist,total_tracks,length,explicit,mode,avg_tempo,instrumentalness,album_sp_pop,artist_genres,artist_sp_pop,feat,feat_mult
153,Desire,Pharoahe Monch,14,3058.83,13,9,84.513,0.00325307,32,"[alternative hip hop, conscious hip hop, east ...",56,1,1
166,Bastard,"Tyler, the Creator",15,3305,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1
186,Be,Common,11,2556.89,11,4,88.738,0.0356825,60,"[alternative hip hop, chicago rap, conscious h...",68,1,1
188,Resurrection,Common,15,3263.13,13,8,91.447,0.400159,55,"[alternative hip hop, chicago rap, conscious h...",68,0,0
202,Infinite,Eminem,11,2274,NaN,NaN,NaN,NaN,NaN,"[detroit hip hop, hip hop, rap]",NaN,1,1
214,Float,Aesop Rock,20,4169.65,1,11,68.476,1.85763,44,"[abstract hip hop, alternative hip hop, consci...",61,0,0
223,/\/\ /\ Y /\,M.I.A.,12,2499.91,0,6,167.833,0.812593,46,"[dance pop, escape room, indietronica, new rave]",77,0,0
224,AIM (Deluxe),M.I.A.,17,3262.32,5,11,130.003,0.358087,58,"[dance pop, escape room, indietronica, new rave]",77,1,0
228,4eva N a Day,Big K.R.I.T.,17,3005,NaN,NaN,NaN,NaN,NaN,"[alternative hip hop, conscious hip hop, dirty...",NaN,0,0
233,It's Better This Way,Big K.R.I.T.,14,2877,NaN,NaN,NaN,NaN,NaN,"[alternative hip hop, conscious hip hop, dirty...",66,1,1


In [54]:
# Re-do merge of AOTY and Spotify data after incorporating corrected Spotify data
sp_feat_df3 = err12_df.combine_first(sp_feat_df2)

,title,artist,total_tracks,length,explicit,mode,avg_tempo,instrumentalness,album_sp_pop,artist_genres,artist_sp_pop,feat,feat_mult
50,Our Pathetic Age,DJ Shadow,26,5500.14,9,13,101.493,8.20911,56,"[alternative hip hop, big beat, downtempo, ele...",66,1,1
51,Liquid Swords,GZA,13,3322.93,3,12,181.804,0.0642611,66,"[alternative hip hop, east coast hip hop, gang...",61,1,1
52,Atrocity Exhibition,Danny Brown,15,2807.54,15,10,83.781,0.0208345,61,"[alternative hip hop, detroit hip hop, escape ...",65,1,1
53,XXX (Deluxe Version),Danny Brown,22,3669.58,22,15,136.95,0.151126,53,"[alternative hip hop, detroit hip hop, escape ...",65,1,0
54,uknowhatimsayin¿,Danny Brown,11,2022.59,11,9,101.852,0.0782908,57,"[alternative hip hop, detroit hip hop, escape ...",65,1,0
55,Old Dominion,Old Dominion,17,3522.48,0,16,76.016,6.36e-05,76,"[contemporary country, country road]",74,0,0
56,The Hybrid - Deluxe Edition,Danny Brown,19,3355.8,19,12,133.027,0.00652215,37,"[alternative hip hop, detroit hip hop, escape ...",65,0,0
57,The Money Store,Death Grips,13,2482.28,13,7,124.885,0.00807759,65,"[alternative hip hop, escape room, experimenta...",64,0,0
58,The Powers That B,Death Grips,18,4844.31,18,16,135.466,0.156674,56,"[alternative hip hop, escape room, experimenta...",64,0,0
59,Ex Military,Death Grips,13,2913.37,11,8,96.915,0.967478,0,"[alternative hip hop, escape room, experimenta...",64,0,0


In [55]:
sp_feat_df3.shape

(1303, 13)

In [59]:
merged_sp_df3 = pd.merge(no_sp_df, sp_feat_df3, left_index=True, right_index=True, how='outer')
merged_sp_df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1303 entries, 0 to 1302
Data columns (total 29 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   title_x                  1303 non-null   object 
 1   artist_x                 1303 non-null   object 
 2   release_date             1303 non-null   object 
 3   format                   1303 non-null   int64  
 4   major_label              1303 non-null   int64  
 5   genres                   1303 non-null   object 
 6   num_tracks               1212 non-null   float64
 7   features_others          1212 non-null   float64
 8   debut_album              1303 non-null   int64  
 9   num_user_ratings         1303 non-null   int64  
 10  critic_score             1135 non-null   float64
 11  artist_critic_score      1228 non-null   float64
 12  artist_user_score        1303 non-null   object 
 13  artist_num_user_ratings  1299 non-null   float64
 14  years_active            

In [60]:
merged_sp_df3.drop(columns=['title_y', 'artist_y'], inplace=True)
merged_sp_df3.head()

,title_x,artist_x,release_date,format,major_label,genres,num_tracks,features_others,debut_album,num_user_ratings,...,length,explicit,mode,avg_tempo,instrumentalness,album_sp_pop,artist_genres,artist_sp_pop,feat,feat_mult
0,To Pimp a Butterfly,Kendrick Lamar,"March 16, 2015",1,1,Hip Hop,16.0,1.0,0,4504,...,4739.61,16,8,86.935,0.00017263,78,"[conscious hip hop, hip hop, rap, west coast rap]",90,1,1
1,"good kid, m.A.A.d. city",Kendrick Lamar,"October 22, 2012",1,0,Hip Hop,11.0,1.0,0,3409,...,4380.27,13,7,133.798,0.0113718,76,"[conscious hip hop, hip hop, rap, west coast rap]",90,1,0
2,DAMN.,Kendrick Lamar,"April 14, 2017",1,0,Hip Hop,14.0,1.0,0,3695,...,3300.97,14,8,174.516,0.0859849,86,"[conscious hip hop, hip hop, rap, west coast rap]",90,1,0
3,Section.80,Kendrick Lamar,"July 2, 2011",1,0,Hip Hop,16.0,0.0,1,1590,...,3272.58,15,7,81.024,0.00868144,73,"[conscious hip hop, hip hop, rap, west coast rap]",90,1,0
4,Kendrick Lamar EP,Kendrick Lamar,"December 29, 2009",0,0,Hip Hop,15.0,1.0,0,106,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
merged_sp_df3.drop(columns=['num_tracks', 'features_others'], inplace=True)
merged_sp_df3.columns

Index(['title_x', 'artist_x', 'release_date', 'format', 'major_label',
       'genres', 'debut_album', 'num_user_ratings', 'critic_score',
       'artist_critic_score', 'artist_user_score', 'artist_num_user_ratings',
       'years_active', 'user_score', 'total_tracks', 'length', 'explicit',
       'mode', 'avg_tempo', 'instrumentalness', 'album_sp_pop',
       'artist_genres', 'artist_sp_pop', 'feat', 'feat_mult'],
      dtype='object')

In [62]:
merged_sp_df3.rename(columns={'title_x': 'title', 'artist_x': 'artist'}, inplace=True)

fin_cols = ['title', 'artist', 'release_date', 'format', 'major_label', 'genres', 'total_tracks',
            'length', 'explicit', 'mode', 'avg_tempo', 'instrumentalness', 'feat', 'feat_mult', 'debut_album',
            'num_user_ratings', 'critic_score', 'artist_critic_score', 'artist_user_score',
            'artist_num_user_ratings', 'years_active', 'album_sp_pop', 'artist_genres', 'artist_sp_pop',
            'user_score']

merged_sp_df3 = merged_sp_df3[fin_cols]
merged_sp_df3.head()

,title,artist,release_date,format,major_label,genres,total_tracks,length,explicit,mode,...,num_user_ratings,critic_score,artist_critic_score,artist_user_score,artist_num_user_ratings,years_active,album_sp_pop,artist_genres,artist_sp_pop,user_score
0,To Pimp a Butterfly,Kendrick Lamar,"March 16, 2015",1,1,Hip Hop,16,4739.61,16,8,...,4504,95.0,89.0,87,13780.0,17.0,78,"[conscious hip hop, hip hop, rap, west coast rap]",90,93
1,"good kid, m.A.A.d. city",Kendrick Lamar,"October 22, 2012",1,0,Hip Hop,13,4380.27,13,7,...,3409,90.0,89.0,87,13780.0,17.0,76,"[conscious hip hop, hip hop, rap, west coast rap]",90,91
2,DAMN.,Kendrick Lamar,"April 14, 2017",1,0,Hip Hop,14,3300.97,14,8,...,3695,93.0,89.0,87,13780.0,17.0,86,"[conscious hip hop, hip hop, rap, west coast rap]",90,82
3,Section.80,Kendrick Lamar,"July 2, 2011",1,0,Hip Hop,15,3272.58,15,7,...,1590,80.0,89.0,87,13780.0,17.0,73,"[conscious hip hop, hip hop, rap, west coast rap]",90,81
4,Kendrick Lamar EP,Kendrick Lamar,"December 29, 2009",0,0,Hip Hop,NaN,NaN,NaN,NaN,...,106,70.0,89.0,87,13780.0,17.0,NaN,NaN,NaN,73


In [63]:
# Pickle final dataframe
outfile = open('fin2_df','wb')
pickle.dump(merged_sp_df3,outfile)
outfile.close()